In [ ]:
import pandas as pd
import requests
from datetime import datetime
import json
import os
from dateutil import parser
import collections

The following code helps to determine whether new data is inserted in api database or not, if so then we have to pull new movies data and put that in database and update tracking parameters. 

In [ ]:
last_page =  requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=821&limit=50').json()
first_page = requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=1&limit=50').json()

In [ ]:
movie_a = last_page['data']['movies'][1]

In [ ]:
movie_a

In [ ]:
movie_a['genres']

In [ ]:
ids = [41763,41713]
ids

In [ ]:
def check_new_movies(ids:list):

    page_number = 1
    movies_list = []
    while True:
        init_page = requests.get(f'https://yts.torrentbay.to/api/v2/list_movies.json?page={page_number}&limit=50').json()

        current_ids = [init_page['data']['movies'][0]['id'],init_page['data']['movies'][-1]['id']]

        if current_ids > ids:
            for movie in init_page['data']['movies']:
                movies_list.append(movie['title'])
        
            page_number = page_number + 1

        else:
            break

    return movies_list

In [ ]:
mov_list = check_new_movies(ids)

In [ ]:
for movie in first_page['data']['movies']:
    print(movie)

Below code is for extracting movie cast based on "IMDB code"

In [10]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [21]:
url = f'https://www.imdb.com/title/tt2065946/fullcredits'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [22]:
table = soup.find('table', attrs={'class' : 'cast_list'})

In [23]:
cast = table.find_all('a')

In [24]:
cast[0].get_text().strip('\n').strip()

''

In [63]:
cast

[<a href="/name/nm1881049/"><img alt="Miho Fujima" class="" height="44" src="https://m.media-amazon.com/images/S/sash/N1QWYSqAfSJV62Y.png" title="Miho Fujima" width="32"/></a>,
 <a href="/name/nm1881049/"> Miho Fujima
 </a>,
 <a href="/name/nm0620267/"><img alt="Yûko Nakamura" class="" height="44" src="https://m.media-amazon.com/images/S/sash/N1QWYSqAfSJV62Y.png" title="Yûko Nakamura" width="32"/></a>,
 <a href="/name/nm0620267/"> Yûko Nakamura
 </a>,
 <a href="/name/nm0913831/"><img alt="Makiko Watanabe" class="loadlate hidden" height="44" loadlate="https://m.media-amazon.com/images/M/MV5BMTY4MDAwNzYxMF5BMl5BanBnXkFtZTcwNjQxNDc3MQ@@._V1_UX32_CR0,0,32,44_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/N1QWYSqAfSJV62Y.png" title="Makiko Watanabe" width="32"/></a>,
 <a href="/name/nm0913831/"> Makiko Watanabe
 </a>]

In [64]:
re.findall(r'title="(.*?)"', str(cast))

['Miho Fujima', 'Yûko Nakamura', 'Makiko Watanabe']

In [62]:
cast_list = []
for actor in cast:
    print(re.findall(r'title="(.*?)"', str(actor)))

['Miho Fujima']
[]
['Yûko Nakamura']
[]
['Makiko Watanabe']
[]


In [ ]:
tuple(cast_list[::3])

## method to get cast list of given imdb movie id

In [65]:
def get_cast(imdb_id : str):
    
    url = f'https://www.imdb.com/title/{imdb_id}/fullcredits'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'class' : 'cast_list'})
    cast = table.find_all('a')   
    
    return re.findall(r'title="(.*?)"', str(cast))

In [69]:
%%time
ac = get_cast('tt2065942')

Wall time: 15.2 s


In [70]:
ac

['Atomic Kitten',
 'Irene Blanco',
 'Sandra Blanco',
 'Miguel Ángel Cabrera',
 'María del Mar',
 'Natalia Estrada',
 'Rosario Flores',
 'Pao González',
 'Sandra Gálvez',
 'Sonia Gálvez',
 'Dioni Martín',
 'Ángeles Muñoz',
 'María Rodríguez',
 'Pedro Ruiz',
 'Gala Évora']